In [1]:
import sys
import pandas as pd
from tra import *

True


In [2]:
import multiprocessing as mp

if __name__ == "__main__":
	mp.set_start_method('spawn', force=True)

	process = []
	for tl in tls:
		p = mp.Process(target=translate, args=(sl, tl))
		p.start()
		process.append(p)

	for p in process:
		p.join()
	

/home2/s4231317/virtual_env/HC/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2023-06-26 10:51:48.664410: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home2/s4231317/virtual_env/HC/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum len

True
en_XX - xh_ZA
['Scammers dupe victims into paying security keeping fees to receive courier parcels supposedly containing large checks or other valuables.'
 ' with two wives receive a host of government benefits upon their their arrival.'
 'Election Day is being split across two separate days this year.'
 'A photograph shows a scarecrow that was made out of a human corpse.'
 "Two men arrested for their part in Turkey's attempted military takeover are CIA agents."]
['Scammers igquma uphinda- phindo lwamaxabiso ozakulayisha iimfihlo zokhuseleko ukufumana iincwadi ezineenkcukacha ze courier eziqulathe iikhasi ezinkulu okanye ezinye ixabiso.'
 'neenamanye amabini abamkela amaninzi avunye lwenkqubo kwixesha elizakufika.'
 'Ikhetho Imini Ikhusi lwahlulekayo lwahlulekayo lwahlulekayo lweentsuku ezimbini ezihlukileyo.'
 'Umfanekiso ubonisa um scarecrow owenziwa khona esuka kumntu.'
 'Amaqela amatsha angaqukwa ngomsebenzi womntu wamthengisi womntu wamthengisi womntu wamthengisi womntu wamth

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForSeq2Seq
from tqdm import tqdm
import torch

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

x_ids = torch.as_tensor([ten["input_ids"] for ten in tqdm(tok_data)])
y_ids = torch.as_tensor([ten["labels"] for ten in tqdm(tok_data)])

print(x_ids.shape, y_ids.shape)

In [ ]:
def convert_to_dict(data, lang):
  d = {}
  for idx, dat in enumerate(data[:6]):
    d[idx] = {lang: data[idx]}

  return d

x_train, x_test, y_train, y_test = train_test_split(x_ids, y_ids, test_size=0.25)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5) # validation

print(x_train.shape, x_test.shape, x_val.shape)

x_train = convert_to_dict(x_train, s_lang)
x_test = convert_to_dict(x_test, s_lang)
x_val = convert_to_dict(x_val, s_lang)

y_train = convert_to_dict(y_train, t_lang)
y_test = convert_to_dict(y_test, t_lang)
y_val = convert_to_dict(y_val, t_lang)

# train = [{"id": str(idx), "translation": {s_lang: x_train, t_lang: y_train}} for idx in ids][0]
# test = [{"id": str(idx), "translation": {s_lang: x_test, t_lang: y_test}} for idx in ids][0]

In [ ]:
class Data(torch.utils.data.Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __getitem__(self, idx):
    item = {key: torch.as_tensor(val[idx]) for key, val in self.x.items()}
    print(item)
    item["labels"] = torch.as_tensor(self.y[idx])
    return item

  def __len__(self):
    return len(self.y)

train = Data(x_train, y_train)
test = Data(x_test, y_test)
val = Data(x_val, y_val)

In [ ]:
!pip install -q evaluate sacrebleu

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir=f"{s_lang}-{t_lang}_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
# sl = "English"
# tl = "French"
# prefix = f"translate {sl} to {tl}: "

# text = prefix + "My name is Eduard"

# inputs = tokenizer(text, return_tensors="pt").input_ids
# outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))